<a href="https://colab.research.google.com/github/EltonGS/BC_DS_projeto_modulo_3/blob/main/Notebooks/modelo_prophet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
# importando as bibliotecas que serão utilizadas
import pandas as pd # biblioteca utilizada para análise dos dados

from fbprophet import Prophet # biblioteca utilizada para previsão de séries temporais

In [47]:
# endereço dados que serão utilizados para o projeto
url_sa = 'https://raw.githubusercontent.com/EltonGS/BC_DS_projeto_modulo_3/main/dados_tratados/dados_sa.csv'

# lendo os dados
dados_sa = pd.read_csv(url_sa)

In [49]:
# transformando os dados da coluna data do dataframe no formato datetime
dados_sa['data'] = pd.to_datetime(dados_sa['data'], format='%Y-%m-%d')

In [50]:
# visualização dos dados
dados_sa.head(3)

,cidade,codigo_cidade,data,semana_epidem,populacao,populacao_2019,is_last,is_repeated,casos_confirmados,casos_por_100k_habitantes,data_atualizacao,taxa_mortalidade,obitos,estado,casos_novos,obitos_novos
0,Santo André,3547809.0,2020-03-16,202012,721368.0,718773.0,False,False,1,0.13863,2020-03-16,0.0,0,SP,1,0
1,Santo André,3547809.0,2020-03-17,202012,721368.0,718773.0,False,False,1,0.13863,2020-03-17,0.0,0,SP,0,0
2,Santo André,3547809.0,2020-03-18,202012,721368.0,718773.0,False,False,6,0.83175,2020-03-18,0.0,0,SP,5,0


O **Prophet** utiliza um padrão para receber os dados e conseguir identificar o que é cada informação. Assim precisamos criar um dataframe com as **colunas 'ds' (datestamp)** e **'y'**. A coluna 'ds' receberá os dados de tempo em um formato identificável pelo **Pandas**, idealmente no formato AAAA-MM-DD para data. A coluna 'y' deve conter valores numéricos e representa a medida que será prevista.

In [51]:
# criação do dataframe que receberá os dados 
df = pd.DataFrame()

In [52]:
# criação das coluna no dataframe df
df['ds'] = dados_sa['data']
df['y'] = dados_sa['casos_novos']

In [54]:
# visualização dos dados no dataframe df
df.head()

,ds,y
0,2020-03-16,1
1,2020-03-17,0
2,2020-03-18,5
3,2020-03-19,-4
4,2020-03-20,1
